In [52]:
import pandas as pd
import locale as lc
import datetime as dt
import math
examplePATH = 'example/'
resultPATH = 'result/'

sale = pd.DataFrame(pd.read_excel('销售单.xlsx', dtype=object))
production = pd.DataFrame(pd.read_excel('生产表.xlsx', dtype=object))
buy = pd.DataFrame(pd.read_excel('采购单.xlsx', dtype=object))

example_sale = pd.DataFrame(pd.read_excel(examplePATH+'销售单.xlsx'))
example_production = pd.DataFrame(pd.read_excel(examplePATH+'生产表.xlsx'))
example_buy=pd.DataFrame(pd.read_excel(examplePATH+'采购单.xlsx'))
reference = pd.DataFrame(pd.read_excel(examplePATH+'目录.xlsx'))
example_stock = pd.DataFrame(pd.read_excel(examplePATH+'库存单.xlsx', dtype=object))

In [53]:
saleC = sale.columns
productionC = production.columns
buyC = buy.columns
categoryC = example_stock.columns
#['日期', '产品名称', '单据类型', '单据编号' '数量', '单价', '金额']

In [54]:
sale.columns = [i.strip() for i in saleC]
production.columns = [i.strip() for i in productionC]
buy.columns = [i.strip() for i in buyC]
example_stock.columns = [i.strip() for i in categoryC]

In [55]:
reference.head()

,商品名称,商品单价,商品起始
0,脂肪族减水剂,5000,20
1,聚羧酸减水剂,5000,20
2,甲醛,5000,20
3,丙酮,5000,20
4,亚纳,5000,20


In [56]:
class setup_(object):
    """
        this class is to deal with first time setup, login the orgin of every products and resources and their number
    
        Attributes:
            reference: stock template
    """
    
    def __init__(self, reference_, example_stock_):
        self.reference_df = pd.DataFrame(reference_)
        self.stock_template = pd.DataFrame(example_stock_)
    
    def first_time(self):
        
        #writer = pd.ExcelWriter(resultPATH+'库存表.xlsx')
        init = self.reference_df
        for idx in range(len(init)):
            df = pd.DataFrame(self.stock_template)
            price = init.loc[idx, '商品单价']
            name = init.loc[idx, '商品名称']
            amount = init.loc[idx, '商品起始']
            
            df.loc[0, '商品名称'] = name
            df.loc[0, '数量'] = amount
            df.loc[0, '单据类型'] = '初始'
            df.loc[0, '单价'] = price
            df.loc[0, '结存数量'] = amount
            df.loc[0, '结存金额'] = price * amount
            df.loc[0, '日期'] = dt.datetime(2017, 4, 1).strftime('%Y-%m-%d')
            
            #df.loc[0, '日期'] = dt.datetime.today().strftime('%Y-%m-%d')
            df.to_excel(resultPATH+name+'.xlsx', index=False)
            #df.to_excel(writer, sheet_name=str(key), index=False)
        #writer.save()
        return None

In [57]:
setup = setup_(reference, example_stock)
setup.first_time()

In [44]:

production['日期'] = pd.to_datetime(production['日期'], errors='coerce')
production = production.sort_values(by='日期')
buy['日期'] = pd.to_datetime(buy['日期'], errors='coerce')
buy = buy.sort_values(by='日期')
sale['日期'] = pd.to_datetime(sale['日期'], errors='coerce')
sale = sale.sort_values(by='日期')

#convert into string
production['日期'] = production['日期'].dt.strftime('%Y-%m-%d')
buy['日期'] = buy['日期'].dt.strftime('%Y-%m-%d')
sale['日期'] = sale['日期'].dt.strftime('%Y-%m-%d')

In [58]:
production = production[pd.notnull(production['单据类型'])]
production.head()

,日期,商品名称,单据类型,单据编号,甲醛,丙酮,亚纳,元明粉,苯酚,葡萄糖酸钠,...,大单体703,过硫酸铵,丙烯酸,丙烯酸乙酯,巯基乙酸,液碱,DX-2促进剂,DX-8调节剂,软水,数量
0,2017-03-21,脂肪族减水剂,成品入库单,0000076,4.248,1.337,1.35,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.402,10.337
1,2017-03-22,脂肪族减水剂,成品入库单,0000077,3.446,1.08,1.09,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.53,8.146
2,2017-03-24,脂肪族减水剂,成品入库单,0000078,6.284,2.18,2.7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.3,20.464
3,2017-03-25,脂肪族减水剂,成品入库单,0000079,6.396,2.18,2.725,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.3,20.601
4,2017-03-26,脂肪族减水剂,成品入库单,0000080,6.396,2.18,2.725,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.3,20.601


In [59]:
a = list(production.loc[0, ['日期', '商品名称', '单据类型', '单据编号', '合成数量']])
a += [123, 22]
math.isnan(a[4]) is False

False

def translation(df, example_df):
    #first we need to translate chinese in english so that we can modify them easier
    header = list(df.columns)
    length = len(header)
    reference = list(example_df.columns)
    reference_length = len(reference)
    header_n = []
    if length == reference_length:
        print('pass')
        #assign new header with english
        for i in range(length):
            if header[i] == reference[i]:
                nh = str('H'+str(i))
                print(nh)
                header_n.append(nh)
                df = df.rename(columns={header[i]: nh})
        #create a dictionary for later use
        header_dic = dict(zip(header_n, header))
        return header_dic
    

writer = pd.ExcelWriter(resultPATH+'库存表.xlsx')
df = pd.DataFrame(pd.read_excel(resultPATH+'库存表.xlsx', sheetname='甲醛'))
df.loc[1, categoryC[0]] = 2259
df.to_excel(writer, sheet_name='甲醛', index=False)
writer.save()

In [60]:
example_stock.columns

Index(['日期', '商品名称', '单据类型', '单据编号', '数量', '单价', '金额', '结存数量', '结存金额'], dtype='object')

In [65]:
class copy_(object):
    '''
        copy data from sell and buy sheet to the stock
        
        Attribute:
            reference: an stock sheet example
            sb_df: the sell or buy dataframe
            st_df: the stock dataframe
    '''
    
    def __init__(self, reference):
        self.ref = reference.columns
        self.r = self.ref[:-2]
    
    
    def sb_find(self, sb_df):
        ca = sb_df.columns
        ref = self.r
        goods = dict(sb_df[ca[1]])
        #file = resultPATH+'库存表.xlsx'
        #writer = pd.ExcelWriter(file)
        for idx, item in goods.items():
            print('{} ...'.format(item))
            df = pd.DataFrame(pd.read_excel(resultPATH+item+'.xlsx', dtype=object))
            for n, i in enumerate(ref):
                string = sb_df.loc[idx, i]
                if 0 < n < 4:
                    string = str(string)
                #else:
                    #string = round(float(string), 2)
                    #string = '{:,.2f}'.format(string)
                df.loc[idx+1, i] = string
            df.to_excel(resultPATH+item+'.xlsx', index=False)
            print('Done!')
            if idx == 1:
                df.head()
        return None
    
    
    def st_find(self, st_df):
        ca = st_df.columns
        ref = self.r
        columns = st_df.columns
        goods = dict(st_df[ca[1]])
        material = columns[4:-1]
        for idx, item in goods.items():
            df = pd.DataFrame(pd.read_excel(resultPATH+item+'.xlsx', dtype=object))
            print(item)
            goods_list = list(st_df.loc[idx, ['日期', '商品名称', '单据类型', '单据编号', '数量']])
            print(goods_list)
            df.loc[len(df), ['日期', '商品名称', '单据类型', '单据编号', '数量']] = goods_list
            df.to_excel(resultPATH+item+'.xlsx', index=False)
            for m in material:
                print(m)
                if math.isnan(st_df.loc[idx, m]) is False:
                    df2 = pd.DataFrame(pd.read_excel(resultPATH+m+'.xlsx', dtype=object))
                    data_list = list(st_df.loc[idx, ['日期', m, '单据编号']]) + ['出库单', m]
                    print(data_list)
                    df2.loc[len(df2), ['日期', '数量', '单据编号', '单据类型', '商品名称']] = data_list
                    df2.to_excel(resultPATH+m+'.xlsx', index=False)
            print('Done!!!!')
        return None

next step convert every date into dataframe and sorted by date, calculate price

In [66]:
copy = copy_(example_stock)

In [67]:
copy.sb_find(sale)
copy.sb_find(buy)

脂肪族减水剂 ...
Done!
脂肪族减水剂 ...
Done!
脂肪族减水剂 ...
Done!
聚羧酸减水剂 ...
Done!
聚羧酸减水剂 ...
Done!
聚羧酸减水剂 ...
Done!
聚羧酸减水剂 ...
Done!
聚羧酸减水剂 ...
Done!
聚羧酸减水剂 ...
Done!
脂肪族减水剂 ...
Done!
聚羧酸减水剂 ...
Done!
聚羧酸减水剂 ...
Done!
聚羧酸减水剂 ...
Done!
聚羧酸减水剂 ...
Done!
甲醛 ...
Done!
丙酮 ...


TypeError: 

In [68]:
copy.st_find(production)

脂肪族减水剂
[Timestamp('2017-03-21 00:00:00'), '脂肪族减水剂', '成品入库单', '0000076', 10.337]
甲醛
[Timestamp('2017-03-21 00:00:00'), 4.248, '0000076', '出库单', '甲醛']
丙酮
[Timestamp('2017-03-21 00:00:00'), 1.337, '0000076', '出库单', '丙酮']
亚纳
[Timestamp('2017-03-21 00:00:00'), 1.35, '0000076', '出库单', '亚纳']
元明粉
苯酚
葡萄糖酸钠
引气剂
改性剂
早强剂
防冻剂
大单体703
过硫酸铵
丙烯酸
丙烯酸乙酯
巯基乙酸
液碱
DX-2促进剂
DX-8调节剂
软水
[Timestamp('2017-03-21 00:00:00'), 3.402, '0000076', '出库单', '软水']
Done!!!!
脂肪族减水剂
[Timestamp('2017-03-22 00:00:00'), '脂肪族减水剂', '成品入库单', '0000077', 8.145999999999999]
甲醛
[Timestamp('2017-03-22 00:00:00'), 3.446, '0000077', '出库单', '甲醛']
丙酮
[Timestamp('2017-03-22 00:00:00'), 1.08, '0000077', '出库单', '丙酮']
亚纳
[Timestamp('2017-03-22 00:00:00'), 1.09, '0000077', '出库单', '亚纳']
元明粉
苯酚
葡萄糖酸钠
引气剂
改性剂
早强剂
防冻剂
大单体703
过硫酸铵
丙烯酸
丙烯酸乙酯
巯基乙酸
液碱
DX-2促进剂
DX-8调节剂
软水
[Timestamp('2017-03-22 00:00:00'), 2.53, '0000077', '出库单', '软水']
Done!!!!
脂肪族减水剂
[Timestamp('2017-03-24 00:00:00'), '脂肪族减水剂', '成品入库单', '0000078', 20.464000000000002]
甲醛
[Timestamp('2017-0

大单体703
[Timestamp('2017-05-05 00:00:00'), 5.475, '0000055', '出库单', '大单体703']
过硫酸铵
[Timestamp('2017-05-05 00:00:00'), 0.033, '0000055', '出库单', '过硫酸铵']
丙烯酸
[Timestamp('2017-05-05 00:00:00'), 0.3825, '0000055', '出库单', '丙烯酸']
丙烯酸乙酯
[Timestamp('2017-05-05 00:00:00'), 0.3, '0000055', '出库单', '丙烯酸乙酯']
巯基乙酸
[Timestamp('2017-05-05 00:00:00'), 0.018, '0000055', '出库单', '巯基乙酸']
液碱
[Timestamp('2017-05-05 00:00:00'), 0.282, '0000055', '出库单', '液碱']
DX-2促进剂
[Timestamp('2017-05-05 00:00:00'), 0.06, '0000055', '出库单', 'DX-2促进剂']
DX-8调节剂
[Timestamp('2017-05-05 00:00:00'), 0.075, '0000055', '出库单', 'DX-8调节剂']
软水
[Timestamp('2017-05-05 00:00:00'), 9.37, '0000055', '出库单', '软水']
Done!!!!
聚羧酸减水剂
[Timestamp('2017-05-07 00:00:00'), '聚羧酸减水剂', '成品入库单', '0000056', 12.8996]
甲醛
丙酮
亚纳
元明粉
苯酚
葡萄糖酸钠
引气剂
改性剂
早强剂
防冻剂
大单体703
[Timestamp('2017-05-07 00:00:00'), 4.4165, '0000056', '出库单', '大单体703']
过硫酸铵
[Timestamp('2017-05-07 00:00:00'), 0.0266, '0000056', '出库单', '过硫酸铵']
丙烯酸
[Timestamp('2017-05-07 00:00:00'), 0.242, '0000056', '

In [133]:
#build timeframe
zhifang = pd.DataFrame(pd.read_excel(resultPATH+'脂肪族减水剂.xlsx'))
zhifang['日期'] = pd.to_datetime(zhifang['日期'], errors='coerce')

In [134]:
#sorted by date
zhifang = zhifang.sort_values(by='日期')
zhifang

,日期,商品名称,单据类型,单据编号,数量,单价,金额,结存数量,结存金额
1,2017-03-28,脂肪族减水剂,销售单,3841.0,12.56,1500.0,18840.0,NaN,NaN
2,2017-03-30,脂肪族减水剂,销售单,3842.0,33.06,1430.0,47275.8,NaN,NaN
0,2017-04-01,脂肪族减水剂,初始,NaN,NaN,NaN,NaN,10000.0,NaN
3,2017-04-25,脂肪族减水剂,销售单,3843.0,12.32,1500.0,18480.0,NaN,NaN
4,2017-05-17,脂肪族减水剂,销售单,3850.0,28.20,1360.0,38352.0,NaN,NaN


如果用户一下子录入一个月的数据，那么系统会把所有的流水按照时间排序，单价会按照动态加权平均，出货时的金额会按照从出货之前所有进货价格的加权平均计算

In [87]:
production_date = dict(production[productionC[0]])
buy_date = dict(buy[buyC[0]])
sale_date = dict(buy[buyC[0]])

In [88]:
def monthly(date):
    #split date by month
    months = {}
    for value in date.values():
        m = value.month
        y = value.year
        time = str(y)+'-'+str(m)
        if time not in list(months.keys()):
            months.update({time: []})
    for idx, d in date.items():
        m = d.month
        y = d.year
        time = str(y)+'-'+str(m)
        for mon in months.keys():
            if mon == time:
                months[mon].append(idx)
    return months

#split the dataframe monthly
production_date_list = [i for i in production_date.values()]
production_date_list[0].m = []
for item in ship.itervalues():
    if item not in ship_list:
        ship_list.append(item)

In [89]:
Pdate = monthly(production_date)
Bdate = monthly(buy_date)
Sdate = monthly(sale_date)

In [90]:
Pdate

{'2017-3': [0, 1, 2, 3, 4, 5, 6],
 '2017-4': [7, 8, 9, 10, 11],
 '2017-5': [12, 13, 14, 15, 16, 17, 18]}

In [ ]:
def sumUp(df, date_dict):
    